Importamos las librerías necesarias

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd

Definimos los parámetros de la representación
- Número de cuerpos
- Factor de velocidad (número de frames que se salta)
- Nombres de los cuerpos

In [ ]:
cuerpos = 10
factor = 100
nombres = ['Sol', 'Mercurio', 'Venus', 'Tierra', 'Marte',
            'Júpiter', 'Saturno', 'Urano', 'Neptuno', 'Plutón']

Creamos las figuras y ejes que vamos a emplear en la representación gráfica

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
fig2, ax2 = plt.subplots(ncols=2)
# Configurar límites y proporciones
ax2[0].set_xlim(0, len(energia_total))
ax2[1].set_xlim(0, len(momento_total))
ax2[0].set_ylim(min(energia_total), max(energia_total))
ax2[1].set_ylim(min(momento_total), max(momento_total))
ax.set_xlim(-50, 50)
ax.set_ylim(-50, 50)
ax.set_aspect('equal')
ax.set_xlabel("x (UA)")
ax.set_ylabel("y (UA)")
ax.set_title("Órbitas de los planetas")
ax.grid(True)
ax.legend(loc='upper right')

Leemos los datos de los ficheros

In [ ]:
posiciones = {}
momentos = {}
energias = {}
for i in range(cuerpos):
    name_pos = f"Datos_cuerpos/cuerpo_{i}.txt"
    name_momento = f"Datos_cuerpos/momentos_{i}.txt"
    name_energia = f"Datos_cuerpos/energia_{i}.txt"
    posiciones[i] = pd.read_csv(name_pos, delimiter='\t', header=0, names=['x', 'y'])
    momentos[i] = pd.read_csv(name_momento, delimiter='\t', header=0)
    energias[i] = pd.read_csv(name_energia, delimiter='\t', header=0)


Definimos la energías y momento angular totales como la suma de todos los datos de energía y momento angular

In [ ]:
energia_total = np.zeros(len(energias[0]))
momento_total = np.zeros(len(momentos[0]))
for i in range(1, cuerpos):
    energia_total += energias[i].values.flatten()
    momento_total += momentos[i].values.flatten()

Inicializamos las trayectorias y posiciones de los planetas en la animación

In [ ]:
# Crear puntos (círculos) para cada planeta
trayectorias = []
planetas = []
for i in range(cuerpos):
    (planeta,) = ax.plot([], [], 'o', label=nombres[i])
    planetas.append(planeta)
    (trayectoria,) = ax.plot([], [], '-')
    trayectorias.append(trayectoria)


# Función de inicialización
def init():
    for planeta, trayectoria in zip(planetas, trayectorias):
        planeta.set_data([], [])
        trayectoria.set_data([], [])
    return trayectorias + planetas

Inicializamos las gráficas de la energía mecánica total y el momento angular total

In [ ]:
energia, = ax2[0].plot([], [], 'r-')
momento, = ax2[1].plot([], [], 'b-')

def init2():
    energia.set_data([], [])
    momento.set_data([], [])
    return energia, momento

Definimos las funciones de actualización de la animación

In [ ]:
def update(frame):
    frame = factor * frame
    for i in range(cuerpos):
        x = posiciones[i].x[frame]
        y = posiciones[i].y[frame]
        planetas[i].set_data([x], [y])  # Usamos listas para que funcione
        trayectorias[i].set_data(posiciones[i].x[:frame], posiciones[i].y[:frame])
    return planetas + trayectorias

def update2(frame):
    frame = factor * frame
    energia.set_data(np.arange(frame), energia_total[:frame])
    momento.set_data(np.arange(frame), momento_total[:frame])
    return energia + momento

Generamos las dos animaciones, tanto para los planetas como para las energías y momentos

In [ ]:
frames = len(posiciones[0])//factor
anim = FuncAnimation(fig, update, init_func=init, frames=frames, blit=True, interval=20)

In [ ]:
frames = len(posiciones[0])//factor
anim2 = FuncAnimation(fig2, update2, init_func=init2, frames=frames, blit=True, interval=20)

In [ ]:
anim.save("animacion_orbitas.mp4", writer="ffmpeg")